In [0]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, RandomSampler
import torch.utils.data as data_utils
from utils import epoch, epoch_robust_bound, epoch_calculate_robust_err, Flatten, generate_kappa_schedule_CIFAR, generate_epsilon_schedule_CIFAR

# Loading dataset

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

In [0]:
BATCH_SIZE = 50
dataset_path = './cifar10'

In [0]:
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)

Files already downloaded and verified


In [0]:
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True}


train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=BATCH_SIZE, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
len(train_loader)

1000

# Model

In [0]:
class CNN_small(torch.nn.Module):
    def __init__(self):

        super(CNN_small, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 4, padding=0, stride=2)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, 4, padding=0, stride=1)
        self.relu2 = nn.ReLU()
        self.flat = Flatten()
        self.linear1 = nn.Linear(32*12*12, 100)
        self.relu3 = nn.ReLU()
        self.last_linear = nn.Linear(100, 10)                
        
    def forward(self, x):
        
        hidden_activations = []
        
        x = self.conv1(x)
        x = self.relu1(x)
        hidden_activations.append(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        hidden_activations.append(x)
        
        x = self.flat(x)
        x = self.linear1(x)
        x = self.relu3(x)
        hidden_activations.append(x)
        
        out = self.last_linear(x)
        hidden_activations.append(out)
        
        return out, hidden_activations

In [0]:
model = CNN_small().to(device)

# Training

In [0]:
opt = optim.Adam(model.parameters(), lr=1e-3)

EPSILON = 8/255
EPSILON_TRAIN = 8/255
epsilon_schedule = generate_epsilon_schedule_CIFAR(EPSILON_TRAIN)
kappa_schedule = generate_kappa_schedule_CIFAR()
batch_counter = 0

print("Epoch   ", "Combined Loss", "MSE Loss", "Test Err", "Test Robust Err", sep="\t")

for t in range(350):
    _, combined_loss, mse_loss = epoch_robust_bound(train_loader, model, epsilon_schedule, device, kappa_schedule, batch_counter, opt)
    
    # check loss and accuracy on test set
    test_err, _ = epoch(test_loader, model, device)
    robust_err = epoch_calculate_robust_err(test_loader, model, EPSILON, device)
    
    batch_counter += 1000
    
    if t == 200:  #decrease learning rate after 200 epochs
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-4
    
    if t == 250:  #decrease learning rate after 250 epochs
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-5
    
    if t == 300:  #decrease learning rate after 300 epochs
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-6
    
    print(*("{:.6f}".format(i) for i in (t, combined_loss, mse_loss, test_err, robust_err)), sep="\t")

Epoch   	Combined Loss	MSE Loss	Test Err	Test Robust Err
0.000000	0.003945	0.000000	0.624300	1.000000
1.000000	0.003526	0.000062	0.578100	1.000000
2.000000	0.003434	0.000096	0.560500	1.000000
3.000000	0.003409	0.000109	0.554300	1.000000
4.000000	0.003293	0.000118	0.552200	0.999900
5.000000	0.003361	0.000114	0.537700	0.997700
6.000000	0.003260	0.000117	0.532400	0.998800
7.000000	0.003251	0.000120	0.530600	0.983000
8.000000	0.003215	0.000122	0.530700	0.979600
9.000000	0.003258	0.000122	0.518600	0.985000
10.000000	0.003226	0.000126	0.520900	0.969900
11.000000	0.003187	0.000132	0.514700	0.974200
12.000000	0.003167	0.000132	0.502300	0.974000
13.000000	0.003154	0.000140	0.506600	0.961300
14.000000	0.003156	0.000142	0.504700	0.952300
15.000000	0.003113	0.000147	0.497900	0.942000
16.000000	0.003112	0.000144	0.499100	0.936300
17.000000	0.003116	0.000144	0.492000	0.935400
18.000000	0.003126	0.000147	0.489300	0.938800
19.000000	0.003145	0.000146	0.487400	0.924700
20.000000	0.003132	0.000155	0.497